<a href="https://colab.research.google.com/github/venturaEffect/ForVideosInstruct-Pix2Pix/blob/main/_ForVideos_InstructPix2Pix_using_diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First you need to download ffmpeg: https://ffmpeg.org/download.htmlWhen to convert video into frames and back.

When you have the video you want to edit. Save it on a folder. On that folder create a '.sh' file. For example 'videotopng.sh'. You will need to know how many frames per second the video has. For that go to properties on the video and you'll get the information. Now, on the 'videotopng.sh' file write:




**ffmpeg -i yourmovie.mp4 -r 30/1 %04d.png**

In this case, the '30/1' means that is 30 frames per second. the '%04d.png' tells to count with four digits. Example: not '1.png' instead '0001.png'. This is useful if you want to use EbSynth on the process also.

Once you have created the frames on the same folder as the video and the 'videotopng.sh' file, upload the folder to your Google Drive. Run the next cell. It will connect this notebook with your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# InstructPix2Pix: Learning to Follow Image Editing Instructions

A demo notebook for [InstructPix2Pix](https://www.timothybrooks.com/instruct-pix2pix/) using [diffusers](https://github.com/huggingface/diffusers). InstructPix2Pix is fine-tuned stable diffusion model which allows you to edit images using language instructions.

<img src='https://instruct-pix2pix.timothybrooks.com/teaser.jpg'/>

Run the next cell


In [ ]:
!pip install -qqq git+https://github.com/huggingface/diffusers.git gradio transformers accelerate safetensors

Load the `StableDiffusionInstructPix2PixPipeline` pipeline

In [ ]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", safety_checker=None)
pipe.to("cuda")
pipe.enable_attention_slicing()

Create an array with all the files with the CORRECT PATH. ALSO IMPORTANT: Change the range from 1 to the number of frames it has your folder. Example: 1 to 2000 frames + 1


In [ ]:
files = []
for i in range(1, TOTAL FRAMES + 1):
       files.append(str("./drive/MyDrive/YOUR-PATH-TO-THE-FOLDER/") + str('{0:04}'.format(i)) + str(".png"))
      
print(files)

Now loop through all the frames and make the trick of instruct-pix2pix changing each frame. write on the prompt how you want the outlook. Remember to create a folder where these new frames will go on your Drive and write the correct path on 'generated_image.save(...)'

In [ ]:
# Iterate through the list and open each image
for file in files:
    with PIL.Image.open(file) as image:
        # Perform operations on the image
        image = PIL.ImageOps.exif_transpose(image)
        image = image.convert("RGB")
        prompt = "WRITE HERE THE PROMPT"
        generated_image = pipe(prompt, image=image, num_inference_steps=20, image_guidance_scale=1).images[0]
        # Save the generated image to Google Drive
        generated_file = file[-8:]
        generated_image.save(f'./drive/MyDrive/FOLDER-WHERE-THE-RESULTS-GOES/{generated_file}')

Once all the frames are done, download the Folder FROM Drive on your computer. On the folder create a file called 'pngtovideo.sh' and write iside the folder:



**ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -pix_fmt yuv420p VIDEO_OUT.mp4**

Here 30 are the frames per second. Change it depending on the frames from the original video. And VIDEO_OUT.mp4 is how the video will be called.

Now, to make the frames into video, open the terminal on the folder where the frames and 'pngtovideo.sh' is and write on the terminal 'sh pngtovideo.sh'.

With this you have the output video.